In [1]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")
import warnings
warnings.filterwarnings("ignore")

___
Enter Team Member Names here (double click to edit):

- Name 1: Allen Hoskins
- Name 2: Brittany Lewandowski
- Name 3: Alexy Morris
- Name 4: Fidel Tamares

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [2]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np
from __future__ import print_function

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.1214353154362896 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset? 
- What does each instance represent? 
- How many classes are in the dataset and what does each class represent?
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
- Is the data represented as a sparse or dense matrix?

___
Enter your answer here:

**Question Set 1**:
- How many instances are in the dataset? 
> **11,314**
- What does each instance represent? 
> **A news article**
- How many classes are in the dataset and what does each class represent?
> **130,107, represent different categories of news articles**
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
> **No, the training data set does not reflect the news today**
- Is the data represented as a sparse or dense matrix?
> **Sparse**

___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [3]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

# Enter distance comparison below for each pair of vectors:
p = 'Placeholder\n'
nl = '\n'
print('\n\nEuclidean Distance\n ab:',euclidean(a,b),nl, 'ac:',euclidean(a,c),nl, 'bc:',euclidean(b,c),nl)
print('Cosine Distance\n ab:',cosine(a,b),nl, 'ac:',cosine(a,c),nl, 'bc:',cosine(b,c),nl)
print('Jaccard Dissimilarity (vectors should be boolean values)\n ab:',jaccard(a>0,b>0),nl, 'ac:',jaccard(a>0,c>0),nl, 'bc:',jaccard(b>0,c>0),nl)

print('\n\nThe most appropriate distance is...')
print ('Cosine. It clearly delineates between topics effectively. B and C are closest (by far),and A is about the same distance from B and C. Euclidean also works well, but not quite as good as "cosine"\n')

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 1.0985184671870858 
 ac: 1.1891405425398236 
 bc: 0.9177794226661623 

Cosine Distance
 ab: 0.6033714113755322 
 ac: 0.7070276149559529 
 bc: 0.4211595343347173 

Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.8821138211382114 
 ac: 0.8754716981132076 
 bc: 0.9087947882736156 



The most appropriate distance is...
Cosine. It clearly delineates between topics effectively. B and C are closest (by far),and A is about the same distance from B and C. Euclidean also works well, but not quite as good as "cosine"



___
# Start of Live Session Assignment
___
<a id="KNN"></a>
<a href="#top">Back to Top</a>
## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* 

**Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

cv = StratifiedShuffleSplit(n_splits=1, test_size = 0.5, train_size=0.5,random_state = 0)

# fill in the training and testing data and save as separate variables
for trainidx, testidx in cv.split(ds.data,ds.target):
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K

K = [1,2,3,4,5,6,7,8,9,10]
for i in K:
    clf = KNeighborsClassifier(n_neighbors=i, weights='uniform', metric='euclidean')
    clf.fit(X_train,y_train)
    pred = clf.predict(X_test)
    print('KNeighboers:',i,'\n','Accuracy Score:',accuracy_score(pred,y_test))

best_k = 1
acc = 0.5973130634611985

#=====================================

print('\nThe best accuracy is:',acc, 'with',best_k,'neighbors')


KNeighboers: 1 
 Accuracy Score: 0.5973130634611985
KNeighboers: 2 
 Accuracy Score: 0.5543574332685168
KNeighboers: 3 
 Accuracy Score: 0.5264274350362382
KNeighboers: 4 
 Accuracy Score: 0.5106947145129928
KNeighboers: 5 
 Accuracy Score: 0.5000883860703553
KNeighboers: 6 
 Accuracy Score: 0.49248718401979846
KNeighboers: 7 
 Accuracy Score: 0.48205762771787164
KNeighboers: 8 
 Accuracy Score: 0.4718048435566555
KNeighboers: 9 
 Accuracy Score: 0.4629662365211243
KNeighboers: 10 
 Accuracy Score: 0.4553650344705674

The best accuracy is: 0.5973130634611985 with 1 neighbors


**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

Enter your answer below:

>**You do not want to use KDTrees with sparse data because of the curse of dimensionality.**

_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. 

**Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [7]:
from sklearn.neighbors import NearestCentroid

metrics = ['l1','l2','cosine']

for d in metrics:
    clf = NearestCentroid(metric=d)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    print('Metric:', d,'Accuracy: ',accuracy_score(pred,y_test))

best_met = 'Cosine'
print('\nThe best distance metric is: ', best_met)

Metric: l1 Accuracy:  0.3028106770372989
Metric: l2 Accuracy:  0.40533851864946085
Metric: cosine Accuracy:  0.487537564079901

The best distance metric is:  Cosine


___
<a id="naive"></a>
<a href="#top">Back to Top</a>
## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions: 

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

Enter you answer here:

> **20 priors need to be parameterized and 2,600,000 conditional probabilities need to be parameterized if we assume there are 130,000 instances as stated above, OR 226,280 conditional probabilities if we assume there are 11,314 features as stated in question one.**


___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: 

**Questions for part 6**: 
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
- What if the data is sparse, does this change your answer? Why or Why not?

Enter your answer here:
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why?
> **If all the instances contain mostly continuous attributes, it would be better to use Gaussian Naive Bayes because of its assumption of having a normal distribution.**
- What if the data is sparse, does this change your answer? Why or Why not?
> **If the data is sparse we would want to use  multinomial naive bayes because our data would not be normally distrbuted.**

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). 

**Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. 

**Question for part 7**: Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)? 

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB


clf_mnb = MultinomialNB(alpha=0.001)
clf_bnb = BernoulliNB(alpha=0.001, binarize=0.02)

# fill in your code here
for clf in [clf_mnb,clf_bnb]:
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    print(accuracy_score(pred,y_test))
    
                         
p2 = 'Naive Bayes classifiers are fast because they assume independence between features'  
p3 = 'Controls the probability of a normal distribution. As alpha increases, the liklihood of a normal distribution increases.'
print('\nThese classifiers are so fast because...\n', p2,'\n')
print('The alpha values control...\n', p3)


0.8886335513523069
0.8741382358140357

These classifiers are so fast because...
 Naive Bayes classifiers are fast because they assume independence between features 

The alpha values control...
 Controls the probability of a normal distribution. As alpha increases, the liklihood of a normal distribution increases.


________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.